In [ ]:
import pandas as pd
import numpy as np

In [45]:
# 读取前一步的结果
GSVA = pd.read_csv('★GSVA_Term_ID对应表.csv',header=0,index_col=0)
aimClass_df = pd.read_csv('▲GO目标分级.csv',header=0,index_col='ID')

In [46]:
GSVA.insert(2,'class2level','')
GSVA

,Term_name_in_gmt,Term_name_in_tree,class2level
ID,,,
GO:0005451,MONOVALENT_CATION_PROTON_ANTIPORTER_ACTIVITY,--,
GO:0006471,PROTEIN_ADP_RIBOSYLATION,--,
GO:0015298,SOLUTE_CATION_ANTIPORTER_ACTIVITY,--,
GO:0018196,PEPTIDYL_ASPARAGINE_MODIFICATION,--,
GO:0030004,CELLULAR_MONOVALENT_INORGANIC_CATION_HOMEOSTASIS,--,
...,...,...,...
GO:0070625,ZYMOGEN_GRANULE_EXOCYTOSIS,zymogen granule exocytosis,
GO:0042589,ZYMOGEN_GRANULE_MEMBRANE,zymogen granule membrane,
GO:0097341,ZYMOGEN_INHIBITION,zymogen inhibition,


In [81]:
aimClass_df

,term_class,term_level,term_name
ID,,,
GO:0016209,MF,2,antioxidant activity
GO:0140657,MF,2,ATP-dependent activity
GO:0038024,MF,2,cargo receptor activity
GO:0003824,MF,2,catalytic activity
GO:0003774,MF,2,cytoskeletal motor activity
...,...,...,...
GO:0023052,BP,2,signaling
GO:0016032,BP,2,viral process
GO:0110165,CC,2,cellular anatomical entity


In [47]:
aimClass = aimClass_df.index.tolist()
print(len(aimClass))
aimClass

62


['GO:0016209',
 'GO:0140657',
 'GO:0038024',
 'GO:0003824',
 'GO:0003774',
 'GO:0140522',
 'GO:0140223',
 'GO:0140912',
 'GO:0060090',
 'GO:0140104',
 'GO:0140677',
 'GO:0140678',
 'GO:0098772',
 'GO:0140313',
 'GO:0140489',
 'GO:0060089',
 'GO:0045735',
 'GO:0140911',
 'GO:0140776',
 'GO:0140777',
 'GO:0140299',
 'GO:0005198',
 'GO:0090729',
 'GO:0140110',
 'GO:0045182',
 'GO:0005215',
 'GO:0044868',
 'GO:0052055',
 'GO:0044092',
 'GO:0044093',
 'GO:0065009',
 'GO:0005488',
 'GO:0031386',
 'GO:0044183',
 'GO:0140691',
 'GO:0044848',
 'GO:0044419',
 'GO:0051703',
 'GO:0065007',
 'GO:0009987',
 'GO:0098754',
 'GO:0032502',
 'GO:0040007',
 'GO:0042592',
 'GO:0002376',
 'GO:0051179',
 'GO:0040011',
 'GO:0008152',
 'GO:0032501',
 'GO:0048519',
 'GO:0043473',
 'GO:0048518',
 'GO:0050789',
 'GO:0000003',
 'GO:0022414',
 'GO:0050896',
 'GO:0048511',
 'GO:0023052',
 'GO:0016032',
 'GO:0110165',
 'GO:0032991',
 'GO:0044423']

### 读取拓扑树

In [ ]:
# 读取树
GO_terms = pd.read_csv('▲GB_termTree.txt',sep = '\t',header=None,index_col=None)
GO_terms.columns = ['ID','Term_name','JSON']
GO_terms.set_index('ID', inplace = True)

In [48]:
GO_terms

,Term_name,JSON
ID,,
GO:1903500,negative regulation of mitotic actomyosin cont...,"{""nodes"":[{""id"":""GO:0061640"",""lbl"":""cytoskelet..."
GO:1903501,positive regulation of mitotic actomyosin cont...,"{""nodes"":[{""id"":""GO:0061640"",""lbl"":""cytoskelet..."
GO:1903504,regulation of mitotic spindle checkpoint,"{""nodes"":[{""id"":""GO:0008150"",""lbl"":""biological..."
GO:1903505,regulation of establishment of actomyosin cont...,"{""nodes"":[{""id"":""GO:0061640"",""lbl"":""cytoskelet..."
GO:1903502,translation repressor complex,"{""nodes"":[{""id"":""GO:0065007"",""lbl"":""biological..."
...,...,...
GO:0051907,S-(hydroxymethyl)glutathione synthase activity,"{""nodes"":[{""id"":""GO:0016846"",""lbl"":""carbon-sul..."
GO:0051908,double-stranded DNA 5'-3' exodeoxyribonuclease...,"{""nodes"":[{""id"":""GO:0090305"",""lbl"":""nucleic ac..."
GO:0198738,cell-cell signaling by wnt,"{""nodes"":[{""id"":""GO:0061355"",""lbl"":""Wnt protei..."


# 对每个go条目计算分属

In [67]:
GO_termsList = GO_terms.index.tolist()
print(len(GO_termsList))

43272


In [79]:
for goID in GSVA.index:
    # 先找到GO_terms中的拓扑
    if goID not in GO_terms.index.tolist():
        GSVA.loc[goID,'class2level'] = 'not_find'
    else:
        ret = []
        topology  = str(GO_terms.loc[goID,'JSON'])
        for level2 in aimClass:
            if level2 in topology:
                ret.append(level2)
        GSVA.loc[goID,'class2level'] = str(ret)
    

In [80]:
GSVA

,Term_name_in_gmt,Term_name_in_tree,class2level
ID,,,
GO:0005451,MONOVALENT_CATION_PROTON_ANTIPORTER_ACTIVITY,--,not_find
GO:0006471,PROTEIN_ADP_RIBOSYLATION,--,not_find
GO:0015298,SOLUTE_CATION_ANTIPORTER_ACTIVITY,--,not_find
GO:0018196,PEPTIDYL_ASPARAGINE_MODIFICATION,--,not_find
GO:0030004,CELLULAR_MONOVALENT_INORGANIC_CATION_HOMEOSTASIS,--,not_find
...,...,...,...
GO:0070625,ZYMOGEN_GRANULE_EXOCYTOSIS,zymogen granule exocytosis,"['GO:0009987', 'GO:0051179']"
GO:0042589,ZYMOGEN_GRANULE_MEMBRANE,zymogen granule membrane,"['GO:0009987', 'GO:0051179', 'GO:0110165']"
GO:0097341,ZYMOGEN_INHIBITION,zymogen inhibition,"['GO:0065007', 'GO:0008152', 'GO:0048519', 'GO..."


In [82]:
# 将所有结果保存
GSVA.to_csv('★各term分级情况.csv', index=True,columns=None)